In [1]:
import os
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.keys import Keys # Importe les clefs pour les touches du clavier
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import time
from time import sleep 

from tqdm import tqdm # librairie permettant d'indiquer le pourcentage de la boucle for


import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
gaming = pd.read_csv('vgsales.csv', index_col=0)
gaming.head(5)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
gaming_parts = np.array_split(gaming, 16)
gaming_parts[15].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
15564,Lovely x Cation 1 & 2,PSV,2015.0,Action,5pb,0.00,0.00,0.02,0.0,0.02
15565,Barbie as the Princess and the Pauper,GBA,2004.0,Adventure,Vivendi Games,0.01,0.00,0.00,0.0,0.02
15566,Blood Bowl: Legendary Edition,PC,2010.0,Sports,Focus Home Interactive,0.00,0.01,0.00,0.0,0.02
15567,Tantei Opera: Milky Holmes,PSP,2010.0,Adventure,Unknown,0.00,0.00,0.02,0.0,0.02
15568,Pirates of the Burning Sea,PC,2008.0,Role-Playing,Koch Media,0.00,0.01,0.00,0.0,0.02


In [4]:
def scroll(value): #Définition d'une fonction pour scroller automatiquement
    for i in range(20): #nombre de micro-scroll à effectuer
        driver.execute_script("window.scrollBy(0, {})".format(value))
        time.sleep(0.1) #Temps entre chaque scroll

In [5]:
#scrapping part 16
gaming_JVC=gaming_parts[15]

# Ouvrir une page Google Chrome
driver = webdriver.Chrome(ChromeDriverManager().install()) 
# créer une attente implicite
driver.implicitly_wait(10)

#Lancer une recherche en utilisant un fichier excel ou csv (en ittérant sur une colonne)

for i, row in tqdm(gaming_JVC.iterrows(), total = len(gaming_JVC)):
    
    # Ouvrir l'adresse du site web
    driver.get('https://www.jeuxvideo.com/tous-les-jeux/')
    try:
        # accepter les cookies
        driver.find_element(By.XPATH, "//*[@id='cmp-main']/button[2]").click()
    except:
        pass
    
    #Ajout d'un temps de deux secondes avant de lancer l'action suivante
    time.sleep(1) 
    
    loupe_recherche=driver.find_element(By.CLASS_NAME, 'searchContainer__1f9uG_')
    loupe_recherche.click()

    try:
        barre_recherche_site=driver.find_element(By.CLASS_NAME,"searchInput__38b75-")
        barre_recherche_site.send_keys(row['Name'])
        barre_recherche_site.send_keys(Keys.ENTER)

    except:
        pass

    try:    
        premier_jeu=driver.find_elements(By.CLASS_NAME,"col")[0]
        premier_jeu.click()

        # cliquer sur la bonne plateforme
        boutons_plateforme=driver.find_elements(By.CLASS_NAME,"gameHeaderBanner__platform")
        boutons_plateforme
    except:
        pass

    try:
        for bouton in boutons_plateforme:
            if str(bouton.text) == str(row['Platform']):
                bouton.click()
            else:
                pass
    except:
        pass

    try:
        test_note=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__gauge")
        gaming_JVC.loc[i, 'test_note_JVC']=test_note.text
    except:
        gaming_JVC.loc[i, 'test_note_JVC']='no_data'
    
    try:
        avis_count=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__count")
        gaming_JVC.loc[i, 'avis_count_JVC']=avis_count.text
    except:
        gaming_JVC.loc[i, 'avis_count_JVC']='no_data'

    try:
        avis_note=driver.find_elements(By.CLASS_NAME, "gameCharacteristicsMain__gauge")[1]
        gaming_JVC.loc[i, 'avis_note_JVC']=avis_note.text
    except:
        gaming_JVC.loc[i, 'avis_note_JVC']='no_data'
    
    scroll(100)

    try:
        game_caracteristique=driver.find_element(By.CLASS_NAME, "gameCharacteristicsDetailed__table")
        gaming_JVC.loc[i, 'description_JVC']=game_caracteristique.text
    except:
        gaming_JVC.loc[i, 'description_JVC']='no_data'
        


100%|██████████| 1037/1037 [12:19:03<00:00, 42.76s/it] 


In [6]:
driver.close()

In [7]:
gaming_JVC.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,test_note_JVC,avis_count_JVC,avis_note_JVC,description_JVC
Rank,,,,,,,,,,,,,,
15564,Lovely x Cation 1 & 2,PSV,2015.0,Action,5pb,0.00,0.00,0.02,0.0,0.02,no_data,no_data,no_data,no_data
15565,Barbie as the Princess and the Pauper,GBA,2004.0,Adventure,Vivendi Games,0.01,0.00,0.00,0.0,0.02,12,(7),13.9,Editeur(s) / Développeur(s) Wayforward Technol...
15566,Blood Bowl: Legendary Edition,PC,2010.0,Sports,Focus Home Interactive,0.00,0.01,0.00,0.0,0.02,17,(4),9.3,Editeur(s) / Développeur(s) Cyanide Studio Foc...
15567,Tantei Opera: Milky Holmes,PSP,2010.0,Adventure,Unknown,0.00,0.00,0.02,0.0,0.02,--,(0),--,Editeur(s) / Développeur(s) Artdink\nSortie Fr...
15568,Pirates of the Burning Sea,PC,2008.0,Role-Playing,Koch Media,0.00,0.01,0.00,0.0,0.02,13,(24),15.5,Editeur(s) / Développeur(s) Daybreak Game Comp...
15569,TV Anime Idolm@ster: Cinderella Girls G4U! Pac...,PS3,2016.0,Misc,Namco Bandai Games,0.00,0.00,0.02,0.0,0.02,no_data,no_data,no_data,no_data
15570,Mary King's Riding School 2,Wii,2009.0,Simulation,Midas Interactive Entertainment,0.00,0.02,0.00,0.0,0.02,no_data,no_data,no_data,no_data
15571,Micro Machines V4,PSP,2006.0,Racing,Codemasters,0.01,0.00,0.00,0.0,0.02,14,(12),14.7,Editeur(s) / Développeur(s) Codemasters Supers...
15572,Entaku no Seito: The Eternal Legend,PSP,2012.0,Role-Playing,Kadokawa Games,0.00,0.00,0.02,0.0,0.02,no_data,no_data,no_data,no_data


In [ ]:
gaming_JVC.to_csv('gaming_JVC_16.csv')